In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import networkx as nx
import numpy as np
import scipy.sparse
import scipy.sparse.csgraph
import time

In [2]:
cd D:\sapienza\Bioinformatics\data

D:\sapienza\Bioinformatics\data


## 1.1

In [3]:
all_curated_genes = pd.read_csv("curated_gene_disease_associations.tsv", delimiter="\t", index_col="geneId", dtype={})

uniprot_map = pd.read_csv("mapa_geneid_4_uniprot_crossref.tsv", delimiter="\t", index_col="GENEID")
# rename for merging
uniprot_map.index = uniprot_map.index.rename("geneId")

hgnc = pd.read_csv("gene_with_protein_product.txt", delimiter="\t", index_col="hgnc_id", dtype={"entrez_id":"str"})
# rename for merging
hgnc = hgnc.rename(columns={"symbol":"geneSymbol"})

C:\Users\Theo\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (38,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


For all genes in the seed gene list, collect the following basic information from the Uniprot:

• official (primary) gene symbol (check if the symbols are updated and approved on the HGNC website; report any 
issue/lack of data/potential misinterpretation)

• Uniprot AC, alphanumeric ‘accession number’ (a.k.a. ’Uniprot entry’)

• protein name (the main one only, do not report the aliases)

• Entrez Gene ID (a.k.a. ‘GeneID’)

• very brief description of its function (keep it very short, i.e. max 20 words)

• notes related to the above information, if any and if relevant

In [4]:
seed_genes = all_curated_genes[all_curated_genes.diseaseId == "C1879321"][['geneSymbol']]

In [5]:
seed_uniprot = pd.merge(seed_genes, uniprot_map, on="geneId", how='left')

In [12]:
seed_uniprot.head()

,geneSymbol,UniProtKB
geneId,,
113,ADCY7,P51828
302,ANXA2,P07355
307,ANXA4,P09525
308,ANXA5,P08758
309,ANXA6,P08133


In [19]:
hgnc.head()

,geneSymbol,name,locus_group,locus_type,status,location,location_sortable,alias_symbol,alias_name,prev_symbol,...,iuphar,kznf_gene_catalog,mamit-trnadb,cd,lncrnadb,enzyme_id,intermediate_filament_db,rna_central_ids,lncipedia,gtrnadb
hgnc_id,,,,,,,,,,,,,,,,,,,,,
HGNC:5,A1BG,alpha-1-B glycoprotein,protein-coding gene,gene with protein product,Approved,19q13.43,19q13.43,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HGNC:24086,A1CF,APOBEC1 complementation factor,protein-coding gene,gene with protein product,Approved,10q11.23,10q11.23,ACF|ASP|ACF64|ACF65|APOBEC1CF,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HGNC:7,A2M,alpha-2-macroglobulin,protein-coding gene,gene with protein product,Approved,12p13.31,12p13.31,FWP007|S863-7|CPAMD5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HGNC:23336,A2ML1,alpha-2-macroglobulin like 1,protein-coding gene,gene with protein product,Approved,12p13.31,12p13.31,FLJ25179|p170,NaN,CPAMD9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HGNC:30005,A3GALT2,"alpha 1,3-galactosyltransferase 2",protein-coding gene,gene with protein product,Approved,1p35.1,01p35.1,IGBS3S|IGB3S,iGb3 synthase|isoglobotriaosylceramide synthase,A3GALT2P,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


One gene (DLEU2) is missing UniProtKB (accession number).

In [13]:
seed_uniprot[seed_uniprot.UniProtKB.isnull()]

,geneSymbol,UniProtKB
geneId,,
8847,DLEU2,NaN


In [6]:
seed_uniprot_hgnc = seed_uniprot.reset_index().merge(
    hgnc[["geneSymbol", "name", "entrez_id", "uniprot_ids"]], 
    on="geneSymbol", how="left")

seed_uniprot_hgnc.head()

,geneId,geneSymbol,UniProtKB,name,entrez_id,uniprot_ids
0,113,ADCY7,P51828,adenylate cyclase 7,113,P51828
1,302,ANXA2,P07355,annexin A2,302,P07355
2,307,ANXA4,P09525,annexin A4,307,P09525
3,308,ANXA5,P08758,annexin A5,308,P08758
4,309,ANXA6,P08133,annexin A6,309,P08133


In [8]:
seed_uniprot_hgnc.entrez_id.astype("category").describe()

count     104
unique    104
top       960
freq        1
Name: entrez_id, dtype: object

There is inconsistence between DisGeNet gene ids and HGNC ids.

4 seed genes from HGNC is missing uniprot ids and enterez ids.

In [9]:
seed_uniprot_hgnc[seed_uniprot_hgnc.geneId != seed_uniprot_hgnc.entrez_id]

,geneId,geneSymbol,source,UniProtKB,name,entrez_id,uniprot_ids
0,113,ADCY7,CTD_human,P51828,adenylate cyclase 7,113,P51828
1,302,ANXA2,CTD_human,P07355,annexin A2,302,P07355
2,307,ANXA4,CTD_human,P09525,annexin A4,307,P09525
3,308,ANXA5,CTD_human,P08758,annexin A5,308,P08758
4,309,ANXA6,CTD_human,P08133,annexin A6,309,P08133
...,...,...,...,...,...,...,...
103,60468,BACH2,CTD_human,Q9BYV9,BTB domain and CNC homolog 2,60468,Q9BYV9
104,79870,BAALC,CTD_human,Q8WXS3,BAALC binder of MAP3K1 and KLF4,79870,Q8WXS3
105,81552,VOPP1,CTD_human,Q96AW1,VOPP1 WW domain binding protein,81552,Q96AW1
106,81848,SPRY4,CTD_human,Q9C004,sprouty RTK signaling antagonist 4,81848,Q9C004


In [10]:
seed_uniprot_hgnc[seed_uniprot_hgnc.uniprot_ids.isnull()]

,geneId,geneSymbol,source,UniProtKB,name,entrez_id,uniprot_ids
44,3005,H1F0,CTD_human,P07305,NaN,NaN,NaN
45,3006,HIST1H1C,CTD_human,P16403,NaN,NaN,NaN
86,8847,DLEU2,CTD_human,NaN,NaN,NaN,NaN
91,10801,SEPT9,CTD_human,Q9UHD8,NaN,NaN,NaN



### gather description for seed genes

In [7]:
function_descriptions=[]
for protein in seed_uniprot_hgnc.UniProtKB:
    if protein:
        page=requests.get('https://www.uniprot.org/uniprot/'+str(protein))
        soup = BeautifulSoup(page.content, 'html.parser')
        try:
            functions = soup.find('meta',attrs={'name': 'description'})['content']
            function_descriptions.append(functions.split('.')[0]) # since we want a short description, we will take the 1st sentence of
            #each instance, practically, untill we see the first dot
        except:
            print('description not found for protein: '+str(protein))
            function_descriptions.append(' ')
            
        
    

description not found for protein: nan


In [8]:
seed_uniprot_hgnc['description']=function_descriptions

In [9]:
seed_uniprot_hgnc.head()

,geneId,geneSymbol,UniProtKB,name,entrez_id,uniprot_ids,description
0,113,ADCY7,P51828,adenylate cyclase 7,113,P51828,Catalyzes the formation of cAMP in response to...
1,302,ANXA2,P07355,annexin A2,302,P07355,Calcium-regulated membrane-binding protein who...
2,307,ANXA4,P09525,annexin A4,307,P09525,Calcium/phospholipid-binding protein which pro...
3,308,ANXA5,P08758,annexin A5,308,P08758,This protein is an anticoagulant protein that ...
4,309,ANXA6,P08133,annexin A6,309,P08133,May associate with CD21


### 1.1 (c)

In [1]:
#seed_uniprot_hgnc.to_csv("task_1.1c.tab", sep="\t")

NameError: name 'seed_uniprot_hgnc' is not defined

In [3]:
seed_uniprot_hgnc=pd.read_csv('task_1.1c.tab',sep="\t")

In [4]:
seed_uniprot_hgnc= seed_uniprot_hgnc.drop('Unnamed: 0',axis=1)

In [5]:
seed_uniprot_hgnc.head()

,geneId,geneSymbol,UniProtKB,name,entrez_id,uniprot_ids,description
0,113,ADCY7,P51828,adenylate cyclase 7,113.0,P51828,Catalyzes the formation of cAMP in response to...
1,302,ANXA2,P07355,annexin A2,302.0,P07355,Calcium-regulated membrane-binding protein who...
2,307,ANXA4,P09525,annexin A4,307.0,P09525,Calcium/phospholipid-binding protein which pro...
3,308,ANXA5,P08758,annexin A5,308.0,P08758,This protein is an anticoagulant protein that ...
4,309,ANXA6,P08133,annexin A6,309.0,P08133,May associate with CD21


-----

## 1.2

### Biogrid PPI database

In [6]:
biogrid = pd.read_csv("BIOGRID-ORGANISM-Homo_sapiens-3.5.179.tab2.txt", delimiter="\t", index_col="#BioGRID Interaction ID")

C:\Users\Theo\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
biogrid['Organism Interactor A'] = biogrid['Organism Interactor A'].astype("category")
biogrid['Organism Interactor B'] = biogrid['Organism Interactor B'].astype("category")

In [8]:
biogrid.shape

(517878, 23)

Filter out only **physical** interaction in **human**.

In [9]:
human_only = biogrid[(biogrid['Organism Interactor A'] == 9606) & 
                     (biogrid['Organism Interactor B'] == 9606) & 
                     (biogrid['Experimental System Type'] == "physical")]

In [10]:
human_only.shape

(476843, 23)

In [11]:
human_only = human_only[['Entrez Gene Interactor A', 
            'Entrez Gene Interactor B',
            'BioGRID ID Interactor A', 
            'BioGRID ID Interactor B',
            'Official Symbol Interactor A', 
            'Official Symbol Interactor B',
            'Experimental System',
            'Experimental System Type',
            'Organism Interactor A', 
            'Organism Interactor B', 
            'Throughput', "Source Database"]]

In [12]:
human_only.head()

,Entrez Gene Interactor A,Entrez Gene Interactor B,BioGRID ID Interactor A,BioGRID ID Interactor B,Official Symbol Interactor A,Official Symbol Interactor B,Experimental System,Experimental System Type,Organism Interactor A,Organism Interactor B,Throughput,Source Database
#BioGRID Interaction ID,,,,,,,,,,,,
103,6416,2318,112315,108607,MAP2K4,FLNC,Two-hybrid,physical,9606,9606,Low Throughput,BIOGRID
117,84665,88,124185,106603,MYPN,ACTN2,Two-hybrid,physical,9606,9606,Low Throughput,BIOGRID
183,90,2339,106605,108625,ACVR1,FNTA,Two-hybrid,physical,9606,9606,Low Throughput,BIOGRID
278,2624,5371,108894,111384,GATA2,PML,Two-hybrid,physical,9606,9606,Low Throughput,BIOGRID
418,6118,6774,112038,112651,RPA2,STAT3,Two-hybrid,physical,9606,9606,Low Throughput,BIOGRID


In [13]:
human_only["Source Database"].astype("category").describe()

count      476843
unique          1
top       BIOGRID
freq       476843
Name: Source Database, dtype: object

Interactions are not symmentrical

Find seed gene interactions 

In [14]:
biogrid_result = human_only.reset_index()[["Official Symbol Interactor A", "Official Symbol Interactor B"]].rename(
    columns={"Official Symbol Interactor A": "symbol_a",
             "Official Symbol Interactor B": "symbol_b"})

In [17]:
hgnc_map = hgnc[["geneSymbol", "uniprot_ids"]]

biogrid_a = biogrid_result.merge(
    hgnc_map.rename(columns={"geneSymbol": "symbol_a"}), how="left").rename(columns={"uniprot_ids": "uniprot_a"})

biogrid_interactions = biogrid_a.merge(
    hgnc_map.rename(columns={"geneSymbol": "symbol_b"}), how="left").rename(columns={"uniprot_ids": "uniprot_b"})

biogrid_interactions["database"] = "Biogrid"

In [18]:
biogrid_interactions.head()

,symbol_a,symbol_b,uniprot_a,uniprot_b,database
0,MAP2K4,FLNC,P45985,Q14315,Biogrid
1,MYPN,ACTN2,Q86TC9,P35609,Biogrid
2,ACVR1,FNTA,Q04771,P49354,Biogrid
3,GATA2,PML,P23769,P29590,Biogrid
4,RPA2,STAT3,P15927,P40763,Biogrid


----

### IID PPI database

In [19]:
columns = [
    "uniprot1",
    "uniprot2",
    "symbol1", 
    "symbol2",
    "methods",
    "pmids", 
    "dbs",
    "evidence type"
]
iid = pd.read_csv("human_annotated_PPIs.txt", delimiter="\t", usecols=columns)

In [22]:
iid['pmids'].replace('-',np.nan,inplace=True)

In [112]:
iid[iid['symbol2']=='Uncharacterized protein']

,uniprot1,uniprot2,symbol1,symbol2,methods,pmids,dbs,evidence type
2954,P27338,Q96BE0,MAOB,Uncharacterized protein,luminescence based mammalian interactome mapping,25036637,intact,exp
7982,P78395,Q96BE0,PRAME,Uncharacterized protein,luminescence based mammalian interactome mapping,25036637,intact,exp
8583,Q14249,Q96BE0,ENDOG,Uncharacterized protein,luminescence based mammalian interactome mapping,25036637,intact,exp
9838,Q05209,Q8N5B1,PTPN12,Uncharacterized protein,two hybrid,15231748,i2d;intact;mint,exp
13885,P49674,Q96BE0,CSNK1E,Uncharacterized protein,luminescence based mammalian interactome mapping,25036637,intact,exp
14186,O94810,Q96BE0,RGS11,Uncharacterized protein,luminescence based mammalian interactome mapping,25036637,intact,exp
25007,P43155,Q96BE0,CRAT,Uncharacterized protein,luminescence based mammalian interactome mapping,25036637,intact,exp
35587,P13639,Q8TE30,EEF2,Uncharacterized protein,anti tag coimmunoprecipitation,24267889,intact,exp
39531,P40424,V9GYH0,PBX1,Uncharacterized protein,-,NaN,iid-ortho,ortho
48626,Q59G26,Q96BE0,"Guanine nucleotide-binding protein, beta-3 sub...",Uncharacterized protein,luminescence based mammalian interactome mapping,25036637,intact,exp


In [23]:
iid = iid[(iid['symbol1'] != 'Uncharacterized protein') & (iid['symbol2']!='Uncharacterized protein')]


In [24]:
iid= iid.dropna(subset=['pmids'])

In [25]:
iid.shape

(940888, 8)

In [7]:
iid.shape

(975877, 8)

In [144]:
iid['evidence type'].unique()

array(['exp', 'pred', 'exp;ortho', 'exp;ortho;pred', 'ortho;pred',
       'exp;pred'], dtype=object)

In [26]:
iid= iid[(iid['evidence type']=='exp')| (iid['evidence type']=='exp;ortho') | (iid['evidence type']=='exp;ortho;pred') |(iid['evidence type']=='exp;pred')]

In [27]:
iid_interactions = iid[["uniprot1", "uniprot2", "symbol1", "symbol2"]].rename(
    columns={"uniprot1": "uniprot_a", 
             "uniprot2": "uniprot_b", 
             "symbol1": "symbol_a", 
             "symbol2": "symbol_b"})
iid_interactions["database"] = "IID"

In [28]:
iid_interactions.head()

,uniprot_a,uniprot_b,symbol_a,symbol_b,database
0,Q9NUX5,Q9NVM4,POT1,PRMT7,IID
3,P62633,Q99729,CNBP,HNRNPAB,IID
6,O43715,O95817,TRIAP1,BAG3,IID
7,Q13285,Q9UKM9,NR5A1,RALY,IID
9,P60174,Q9H0W5,TPI1,CCDC8,IID


### 1.2 (b)

In [29]:
iid_interactions.to_csv("interactions_iid.tab", sep="\t")
biogrid_interactions.to_csv("interactions_biogrid.tab", sep="\t")

## 1.2(c)

In [30]:
biogrid_interactions= biogrid_interactions.dropna(subset=['uniprot_a','uniprot_b'])
biogrid_interactions= biogrid_interactions.drop_duplicates(subset=['uniprot_a','uniprot_b'])

### IID dataset further preprocessing, since it must be modified to be used in the enrich service

In [31]:
iid_interactions= iid_interactions.dropna(subset=['uniprot_a','uniprot_b'])
iid_interactions= iid_interactions.drop_duplicates(subset=['uniprot_a','uniprot_b'])

In [32]:
iid_interactions['symbol_a']=iid_interactions['symbol_a'].apply(lambda x:x.split(';')[0]) # some columns have multiple values, devided by semicolon
iid_interactions['symbol_b']=iid_interactions['symbol_b'].apply(lambda x:x.split(';')[0])

In [33]:
iid_interactions.shape

(330785, 5)

### Lets cut all the strange values, that cant be imported to enrich software. All cells must have have <2 words

In [34]:
print(iid_interactions[iid_interactions['symbol_a'].apply(lambda x: len(x.split(' ')) >= 2)]) # same process must be done for symbol b as well

       uniprot_a uniprot_b                                           symbol_a  \
1900      Q8NCK5    Q9UKS6              cDNA FLJ90195 fis, clone MAMMA1001310   
2433      G2XKQ0    Q59GP6                   Small ubiquitin-related modifier   
9794      B3KR50    P00533  cDNA FLJ33691 fis, clone BRAWH2002976, highly ...   
13283     A8K251    O95816                                      cDNA FLJ77884   
14998     A9UF07    O15357                                     BCR/ABL fusion   
15476     B2R7W4    P20963  cDNA, FLJ93632, highly similar to Homo sapiens...   
19402     B3KX72    Q5U5Q3  cDNA FLJ44920 fis, clone BRAMY3011501, highly ...   
22656     B2RCZ4    Q9NPB6                                   Protein kinase C   
24045     A8K940    Q8WWW0  cDNA FLJ77630, highly similar to Homo sapiens ...   
25554     Q53G83    Q9UBX2                       Ribosomal protein S3 variant   
25818     B2R8Y4    Q12933  cDNA, FLJ94117, highly similar to Homo sapiens...   
28874     B4DNW0    O00189  

In [35]:
iid_interactions=iid_interactions[iid_interactions['symbol_a'].apply(lambda x: len(x.split(' ')) == 1)] # same process must be done for symbol b as well

In [36]:
iid_interactions.shape

(330394, 5)

In [37]:
iid_interactions=iid_interactions[iid_interactions['symbol_b'].apply(lambda x: len(x.split(' ')) == 1)] # same process must be done for symbol b as well

In [38]:
iid_interactions.shape

(330125, 5)

In [39]:
iid_interactions[iid_interactions['symbol_a']=='IGL@'] # dont know if its valid symbol, but according to enrich service its not

,uniprot_a,uniprot_b,symbol_a,symbol_b,database
145774,Q6PIQ7,Q9UMX0,IGL@,UBQLN1,IID
389718,Q8N355,Q9UMX0,IGL@,UBQLN1,IID
466491,Q6GMX4,Q9NRR5,IGL@,UBQLN4,IID
920166,Q6PIK1,Q9UMX0,IGL@,UBQLN1,IID


In [40]:
iid_interactions=iid_interactions[iid_interactions['symbol_a'].apply(lambda x: str(x) !='IGL@')] # same process must be done for symbol b as well

In [41]:
iid_interactions=iid_interactions[iid_interactions['symbol_b'].apply(lambda x: str(x) !='IGL@')] # same process must be done for symbol b as well

In [42]:
iid_interactions.shape

(330116, 5)

In [47]:
print(biogrid_interactions[biogrid_interactions['uniprot_a'].apply(lambda x: len(x.split('|')) >= 2)])
# we see that some interactions have multiple values for uniptot symbol. we will keep only the 1st value in this case


       symbol_a  symbol_b                    uniprot_a uniprot_b database
1240      DDIT3  HSD17B14                P0DPQ6|P35638    Q9BPX1  Biogrid
2583      DDIT3     MCMBP                P0DPQ6|P35638    Q9BTE3  Biogrid
3106      DDIT3      SPOP                P0DPQ6|P35638    O43791  Biogrid
3629      DDIT3      BATF                P0DPQ6|P35638    Q16520  Biogrid
3871       BBC3      BCL2                Q96PG8|Q9BXH1    P10415  Biogrid
3872       BBC3    BCL2L1                Q96PG8|Q9BXH1    Q07817  Biogrid
3873       BBC3      MCL1                Q96PG8|Q9BXH1    Q07820  Biogrid
4978       GNAS     FSCN1  O95467|P63092|P84996|Q5JWF2    Q16658  Biogrid
5434       GNAS     PAICS  O95467|P63092|P84996|Q5JWF2    P22234  Biogrid
5611       GNAS    RUVBL1  O95467|P63092|P84996|Q5JWF2    Q9Y265  Biogrid
5732       GNAS     SHMT2  O95467|P63092|P84996|Q5JWF2    P34897  Biogrid
6279     CDKN2A      CDK4                P42771|Q8N726    P11802  Biogrid
6280     CDKN2A      CDK6             

In [65]:
biogrid_interactions['uniprot_a']=biogrid_interactions['uniprot_a'].apply(lambda x: x.split('|')[0] if(len(x.split('|')))>1 else x)


In [67]:
biogrid_interactions['uniprot_b']=biogrid_interactions['uniprot_b'].apply(lambda x: x.split('|')[0] if(len(x.split('|')))>1 else x)


In [68]:
seed_uniprot_hgnc=seed_uniprot_hgnc.dropna(subset=['UniProtKB'])

In [69]:
seed_genes_list= list(seed_uniprot_hgnc.UniProtKB)

In [70]:
len(seed_genes_list)

107

In [71]:
def results_1_2(dataframe):
    set1 =set(pd.concat([dataframe['uniprot_a'],dataframe['uniprot_b']]))
    print('number of seed genes found in database: '+dataframe.iloc[0][-1] +' = ' +str(len(set1.intersection(set(seed_genes_list)))))
    G=nx.Graph()
    Graph=nx.from_pandas_edgelist(dataframe[['uniprot_a','uniprot_b']],create_using=G,source='uniprot_a',target='uniprot_b')
    interacting_proteins=set()
    for node in Graph.nodes():
        if node in seed_genes_list:
            for neighbour in Graph.neighbors(node):
                interacting_proteins.add(neighbour)
    print('total number of interacting proteins found in database: '+dataframe.iloc[0][-1]+' = '  + str(len(interacting_proteins)))
    print('total number of interactions in Database : '+dataframe.iloc[0][-1] +' = '+str(len(Graph.edges)))
    

In [72]:
results_1_2(biogrid_interactions)

number of seed genes found in database: Biogrid = 101
total number of interacting proteins found in database: Biogrid = 5037
total number of interactions in Database : Biogrid = 308003


In [73]:
results_1_2(iid_interactions)

number of seed genes found in database: IID = 107
total number of interacting proteins found in database: IID = 4745
total number of interactions in Database : IID = 330116


###  1.3 (a)

In [75]:
seed_genes_interactome_biogrid= biogrid_interactions[(biogrid_interactions['uniprot_a'].isin(seed_genes_list)) | (biogrid_interactions['uniprot_b'].isin(seed_genes_list))]

In [76]:
seed_genes_interactome_iid= iid_interactions[(iid_interactions['uniprot_a'].isin(seed_genes_list)) | (iid_interactions['uniprot_b'].isin(seed_genes_list))]

In [77]:
seed_genes_interactome=pd.concat([seed_genes_interactome_biogrid,seed_genes_interactome_iid],sort=False)

In [204]:
seed_genes_interactome[seed_genes_interactome['symbol_a']=='SEPT9'] # load the columns as text from microsoft excel, so this value is not datetime

,symbol_a,symbol_b,uniprot_a,uniprot_b,database
103670,SEPT9,JPT1,Q9UHD8,Q9UK76,IID
491299,SEPT9,SRRM2,Q9UHD8,Q9UQ35,IID
617605,SEPT9,IKBKG,Q9UHD8,Q9Y6K9,IID
871702,SEPT9,TRAF6,Q9UHD8,Q9Y4K3,IID


In [78]:
seed_genes_interactome.shape

(19802, 5)

In [84]:
seed_genes_interactome.to_csv('seed_genes_interactome',sep='\t')

### 1.3 (b)

In [90]:
def find_neighbor_proteins(dataframe):
    G=nx.Graph()
    Graph=nx.from_pandas_edgelist(dataframe[['uniprot_a','uniprot_b']],create_using=G,source='uniprot_a',target='uniprot_b')
    interacting_proteins=set()
    for node in Graph.nodes():
        if node in seed_genes_list:
            for neighbour in Graph.neighbors(node):
                interacting_proteins.add(neighbour)
    return interacting_proteins
    

In [91]:
biogrid_interacting_proteins=find_neighbor_proteins(biogrid_interactions)

In [92]:
iid_interacting_proteins=find_neighbor_proteins(iid_interactions)

In [93]:
union_interactome_biogrid= biogrid_interactions[(biogrid_interactions['uniprot_a'].isin(biogrid_interacting_proteins)) | (biogrid_interactions['uniprot_b'].isin(biogrid_interacting_proteins))]

In [94]:
union_interactome_iid= iid_interactions[(iid_interactions['uniprot_a'].isin(iid_interacting_proteins)) | (iid_interactions['uniprot_b'].isin(iid_interacting_proteins))]

In [95]:
union_interactome=pd.concat([union_interactome_biogrid,union_interactome_iid],sort=False)

In [96]:
union_interactome.to_csv('union_interactome',sep='\t')

### 1.3 (c)

In [97]:
confirmed_proteins=biogrid_interacting_proteins.intersection(iid_interacting_proteins)

In [98]:
intersection_interactome_biogrid= biogrid_interactions[(biogrid_interactions['uniprot_a'].isin(confirmed_proteins)) | (biogrid_interactions['uniprot_b'].isin(confirmed_proteins))]

In [99]:
intersection_interactome_iid= iid_interactions[(iid_interactions['uniprot_a'].isin(confirmed_proteins)) | (iid_interactions['uniprot_b'].isin(confirmed_proteins))]

In [100]:
intersection_interactome=pd.concat([intersection_interactome_biogrid,intersection_interactome_iid],sort=False)

In [101]:
intersection_interactome=intersection_interactome.drop('database',axis=1)

In [102]:
intersection_interactome.to_csv('intersection_interactome',sep='\t')

In [103]:
seed_genes_interactome.shape

(19802, 5)

In [104]:
union_interactome.shape

(573914, 5)

### Part 2.1

In [4]:
seed_genes_interactome=pd.read_csv('seed_genes_interactome',delimiter='\t')
seed_genes_interactome= seed_genes_interactome.drop('Unnamed: 0',axis=1)

In [8]:
seed_genes_interactome.head()

,symbol_a,symbol_b,uniprot_a,uniprot_b,database
0,GATA2,PML,P23769,P29590,Biogrid
1,RPA2,STAT3,P15927,P40763,Biogrid
2,CSF1R,GRB2,P07333,P62993,Biogrid
3,KDR,ANXA5,P35968,P08758,Biogrid
4,SUMO1,FAS,P63165,P25445,Biogrid


In [4]:
union_interactome=pd.read_csv('union_interactome',delimiter='\t')
union_interactome= union_interactome.drop('Unnamed: 0',axis=1)

In [10]:
union_interactome.head()

,symbol_a,symbol_b,uniprot_a,uniprot_b,database
0,MAP2K4,FLNC,P45985,Q14315,Biogrid
1,MYPN,ACTN2,Q86TC9,P35609,Biogrid
2,GATA2,PML,P23769,P29590,Biogrid
3,RPA2,STAT3,P15927,P40763,Biogrid
4,ARF1,GGA3,P84077,Q9NZ52,Biogrid


In [3]:
intersection_interactome=pd.read_csv('intersection_interactome',delimiter='\t')
intersection_interactome= intersection_interactome.drop('Unnamed: 0',axis=1)

In [12]:
intersection_interactome.head()

,symbol_a,symbol_b,uniprot_a,uniprot_b
0,MAP2K4,FLNC,P45985,Q14315
1,MYPN,ACTN2,Q86TC9,P35609
2,GATA2,PML,P23769,P29590
3,RPA2,STAT3,P15927,P40763
4,ARF1,GGA3,P84077,Q9NZ52


In [4]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

In [75]:
def calculate_centralization(dataframe):
    G=nx.Graph()
    Graph=nx.from_pandas_edgelist(dataframe[['uniprot_a','uniprot_b']],create_using=G,source='uniprot_a',target='uniprot_b')
    density=2*len(Graph.edges())/(len(Graph.nodes())*(len(Graph.nodes())-1))
    
    #print(density)
    #print(max([val for (node, val) in G.degree()])/(len(Graph.nodes())-1))
    print('centralization is : '+ str(max([val for (node, val) in G.degree()])/(len(Graph.nodes())-1)-density))
 

In [ ]:
def calculate_global_measures(dataframe):
    G=nx.Graph()
    Graph=nx.from_pandas_edgelist(dataframe[['uniprot_a','uniprot_b']],create_using=G,source='uniprot_a',target='uniprot_b')
    print('NAME OF NETWORK IS: '+get_df_name(dataframe))
    print('number of nodes:  '+str(len(Graph.nodes())),'number of edges: '+str(len(Graph.edges())))
    print('sizes of connected components: '+str([len(c)  for c in nx.connected_components(Graph)]))
    print('number of isolated nodes: '+''.join([ len(c)  for c in nx.isolates(Graph)]))
    print('average path length is : '+ str(nx.average_shortest_path_length(Graph)))
    print('average degree is: '+str(mean(list(nx.average_degree_connectivity(Graph).values()))))
    print('average clustering coeficcient is:  '+ str(nx.average_clustering(Graph)))
    print('diameter of network is: '+str(nx.diameter(Graph)))
    calculate_centralization(dataframe)
    

In [ ]:
calculate_global_measures(seed_genes_interactome)

In [ ]:
calculate_global_measures(union_interactome)

In [ ]:
calculate_global_measures(intersection_interactome)

In [35]:
def LCC_measures(dataframe):
    G=nx.Graph()
    Graph=nx.from_pandas_edgelist(dataframe[['uniprot_a','uniprot_b']],create_using=G,source='uniprot_a',target='uniprot_b')
    largest_cc = max(nx.connected_components(G), key=len)
    print(len(largest_cc))
    print(nx.is_connected(Graph))


In [36]:
LCC_measures(seed_genes_interactome)

5951
True


In [37]:
LCC_measures(union_interactome)

16820
True


In [38]:
LCC_measures(intersection_interactome)

16449
True


### So, we have only 1 connected component, compiled by all the nodes of the graph, for all the interactomes
### This means that LCC I= intersection intercome and LCC U= union interactome

In [5]:
def closeness_centrality_list(dataframe):
    G=nx.Graph()
    Graph=nx.from_pandas_edgelist(dataframe[['uniprot_a','uniprot_b']],create_using=G,source='uniprot_a',target='uniprot_b')
    A = nx.adjacency_matrix(Graph).tolil()
    D = scipy.sparse.csgraph.floyd_warshall( \
             A, directed=False, unweighted=True)
    n = D.shape[0]
    closeness_centrality = {}
    for r in range(0, n):
    
        cc = 0.0
    
        possible_paths = list(enumerate(D[r, :]))
        shortest_paths = dict(filter( \
            lambda x: not x[1] == np.inf, possible_paths))

        total = sum(shortest_paths.values())
        n_shortest_paths = len(shortest_paths) - 1.0
        if total > 0.0 and n > 1:
            s = n_shortest_paths / (n - 1)
            cc = (n_shortest_paths / total) * s
        closeness_centrality[r] = cc
    return closeness_centrality.values()

In [7]:
def LCC_local_measures(dataframe):
    start_time = time.time()
    G=nx.Graph()
    Graph=nx.from_pandas_edgelist(dataframe[['uniprot_a','uniprot_b']],create_using=G,source='uniprot_a',target='uniprot_b')
    betweeness_threshold=int(len(Graph.nodes())//3)
    df=pd.DataFrame(columns=['Name','Node degree','Betweeness centrality','Eigenvector centrality','Closeness centrality','ratio betweeness'])
    df['Name']=[node for node in Graph.nodes()]
    print("--- %s second(s) for reading and Name ---" % (int(time.time() - start_time)))

    df['Node degree']=[Graph.degree(n) for n in Graph.nodes()]
    start_time = time.time()

    df['Betweeness centrality']=nx.betweenness_centrality(Graph,k=betweeness_threshold).values()
    print("--- %s minutes for Betweeness ---" % (int((time.time() - start_time)/60)))
    start_time = time.time()

    df['Eigenvector centrality']=nx.eigenvector_centrality(Graph,max_iter=100,tol=1e-06).values()
    print("--- %s minutes for Eigenvector ---" % (int((time.time() - start_time)/60)))
    start_time = time.time()

    df['Closeness centrality']=closeness_centrality_list(dataframe)
    print("--- %s minutes for closeness ---" % (int((time.time() - start_time)/60)))
    
    df['ratio betweeness']=df['Betweeness centrality']/ df['Node degree']
    
    
    df.to_csv('LCC_'+str(get_df_name(dataframe)[0]).capitalize(),sep='\t')


    
        
        

In [22]:
LCC_local_measures(union_interactome)

--- 1 second(s) for reading and Name ---
--- 55 minutes for Betweeness ---
--- 0 minutes for Eigenvector ---


In [8]:
LCC_local_measures(intersection_interactome)

--- 0 second(s) for reading and Name ---
--- 18 minutes for Betweeness ---
--- 0 minutes for Eigenvector ---
--- 62 minutes for closeness ---
